# Seyed Mohammad Amin Atyabi - 830402014 - HW 3

In [ ]:
def evaluate_expression(formula, value):
    expression = formula
    for v in value.axes[-1].values:
        expression = expression.replace(f'_{str(v)}_', str(value[v]))
    return eval(expression)


def train_model(data):
    P = data.loc[data['y'] == True]
    h = 'False'
    while len(P) != 0:
        r = 'True'
        r_features = set()
        N = data.loc[data['y'] == False]
        while len(N) != 0:
            if set(data.axes[-1].values).issubset(r_features):
                break
            vj = {}
            for f in data.axes[-1].values:
                if f == 'y':
                    continue
                if f not in r_features:
                    n_plus = 0
                    for i, p in P.iterrows():
                        if p['y'] == evaluate_expression(r + f' and _{f}_', p):
                            n_plus += 1
                    n_minus = 0
                    for i, n in N.iterrows():
                        if n['y'] == evaluate_expression(r + f' and _{f}_', n):
                            n_minus += 1
                    heuristic = n_plus / max(float(n_minus), 0.0001)
                    vj[f] = heuristic
            fj = sorted(vj.items(), key=lambda item: item[1])[-1][0]
            r_features.add(fj)
            n_drop = []
            for i, n in N.iterrows():
                if not n[fj]:
                    n_drop.append(i)
            if len(n_drop) > 0:
                r = r + f' and _{fj}_'
                N = N.drop(index=n_drop, inplace=False)
        h = h + f' or ({r})'
        covered = []
        for i, p in P.iterrows():
            if p['y'] == evaluate_expression(r, p):
                covered.append(i)
        if len(covered) == 0:
            break
        P = P.drop(index=covered, inplace=False)
    return h


def evaluate_model(data, model):
    count = 0
    for _, d in data.iterrows():
        res = (evaluate_expression(model, d) == d['y'])
        if not res:
            count += 1
    return count


def latex_expression(model):
    return model.replace('False or ', '').replace('True and ', '') \
        .replace('_f', 'f_{').replace('_ ', '} ').replace('_)', '})') \
        .replace('and', r'\wedge').replace('or', r'\vee')

In [ ]:
import pandas
from scipy import io as sio
from sklearn.model_selection import train_test_split

data = pandas.DataFrame(sio.loadmat('Data/DNFData.mat').get('Data'))

columns = []
for i in range(len(data.iloc[-1]) - 1):
    columns.append(f'f{i + 1}')
columns.append('y')

data = data.astype(bool)
data = pandas.DataFrame(data.values, columns=columns)

train, test = train_test_split(data, test_size=0.5)

In [ ]:
from IPython.display import display, Markdown

samples = list(range(20, 1001, 20))
errors = []

for s in samples:
    model = train_model(train.sample(s))
    error = evaluate_model(train, model)
    errors.append(error)
    display(Markdown(f"""
    Number of Samples = {s}
    Error = {error}
    $$ {latex_expression(model)} $$
    """))
    if error == 0:
        display(Markdown(f"""
        *Best model*
        
        {model}
        """))

In [ ]:
from matplotlib import pyplot as plt

plt.plot(samples, errors, color='c')
plt.xlabel('Number of samples')
plt.ylabel('$ Error $')
plt.grid()
plt.show()